In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


# Description

# Modules and Global Variables

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, 
    TrainingArguments, Trainer,
)

import torch
import wandb

import datasets
import evaluate

import numpy as np
import pandas as pd

import os
import re
import random

from module.args import *

print_torch_info()

torch.__version__: 1.7.1
torch.cuda.is_available(): True
NGPU: 1


In [3]:
### labels

labels = pc_labels

label2id = {k: i for i, k in enumerate(labels)}
id2label = {i: k for i, k in enumerate(labels)}
num_labels = len(labels)

print(label2id)
print(id2label)

{'positive': 0, 'negative': 1, 'neutral': 2}
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [4]:
### paths and names

PROJECT_NAME = 'aspect_sentiment_classification_multi'
RUN_ID = 'uncleaned_v23_run_1'

DATA_V = 'uncleaned_v23'
DATA_T = 'pc' # ce or pc or pc_binary
AUGMENTATION = False
AUG_NAME = ''

MAX_LEN = False

model_checkpoint = 'klue_roberta_base_mlm_fine_tuned'

notebook_name = 'asc_train_m.ipynb'

### fixed

model_name = re.sub(r'[/-]', r'_', model_checkpoint).lower()
run_name = f'asc_{model_name}_{RUN_ID}'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', RUN_ID)
NOTEBOOK_PATH = os.path.join(ROOT_PATH, notebook_name)

augornot = f'_{AUG_NAME}' if AUGMENTATION is True else ''
TRAIN_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_train{augornot}.csv')
EVAL_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_dev.csv')

!mkdir -p {SAVE_PATH}

In [5]:
print_paths(PROJECT_NAME, model_checkpoint, DATA_V, SAVE_PATH, NOTEBOOK_PATH, TRAIN_DATA_PATH, EVAL_DATA_PATH)
print()
print_args()

PROJECT_NAME: aspect_sentiment_classification_multi

model_checkpoint: klue_roberta_base_mlm_fine_tuned
DATA_V: uncleaned_v23

SAVE_PATH: ./training_results/uncleaned_v23_run_1 exists.
NOTEBOOK_PATH: ./asc_train_m.ipynb exists.

TRAIN_DATA_PATH: ./dataset/uncleaned_v23/pc_train.csv exists.
EVAL_DATA_PATH: ./dataset/uncleaned_v23/pc_dev.csv exists.

per_device_train_batch_size: 24
per_device_eval_batch_size: 24
learning_rate: 2e-05
num_train_epochs: 12


# WandB Configuration

In [6]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

env: WANDB_PROJECT=aspect_sentiment_classification_multi
env: WANDB_NOTEBOOK_NAME=./asc_train_m.ipynb
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Load Model, Tokenizer, and Collator

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, label2id=label2id, id2label=id2label, num_labels=num_labels
)

if MAX_LEN:
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=MAX_LEN)
else:
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at klue_roberta_base_mlm_fine_tuned were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue_roberta_base_mlm_fine_tuned and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 

In [8]:
train_path = f'./dataset/{DATA_V}/raw_train.csv'
dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
test_path = f'./dataset/{DATA_V}/raw_test.csv'
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

print(len(tokenizer))
tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form]).to_frame().drop_duplicates()
tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
new_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
new_tokens = set(list(new_tokenizer.vocab.keys())) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
print(len(new_tokenizer))
print(len(tokenizer))
model.resize_token_embeddings(len(tokenizer))

print(len(new_tokens))
print(new_tokens)

32523



3213
32616
93
{'겆', '욘', '옻', '##뫄', '🎈', '😰', '妆', '🌷', '☕', '😖', '놋', '##냑', '👱', '##🙃', '숄', '##😟', '##챠', '##👱', '💗', '莎', '##ᴄ', '##샨', '뫄', '☀', '##푠', '훅', '➡', '😟', '😞', 'ʟ', '##😩', 'ᴋ', '##🐦', '##뽈', '##꺅', '푠', '씰', '🚓', '##🐾', '##숄', '##ʟ', '뽈', '##ᴋ', '##퀼', '品', '뎁', '🍏', '丽', '╹', '훓', '##뎁', '똬', '😩', '💁', '##🚓', '##댱', '耳', '🔊', '##ﾞ', '##욘', 'ᴄ', '챠', '##🌟', '댱', '🌞', '뗐', '😅', '🔽', 'ﾞ', '熊', '韩', '🐾', '国', '##╹', '냑', '❀', 'ﾉ', '##겆', '🌼', '🐦', '##😨', '伊', '##씰', '😤', '##🎈', '##똬', '샨', 'ヾ', '✈', '봊', '长', '🍊', '😨'}


In [9]:
model.config.label2id, model.config.id2label, model.num_labels

({'positive': 0, 'negative': 1, 'neutral': 2},
 {0: 'positive', 1: 'negative', 2: 'neutral'},
 3)

# Define Metrics

In [10]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [11]:
### multi
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_metric.compute(references=labels, predictions=predictions)['accuracy']
    f1_positive, f1_negative, f1_neutral = tuple(f1_metric.compute(references=labels, predictions=predictions, average=None, labels=[0,1,2])['f1'])
    f1_macro = f1_metric.compute(references=labels, predictions=predictions, average='macro')['f1']
    f1_micro = f1_metric.compute(references=labels, predictions=predictions, average='micro')['f1']
    
    return {'accuracy': accuracy, 
            'f1_positive': f1_positive, 'f1_negative': f1_negative, 'f1_neutral': f1_neutral, 
            'f1_macro': f1_macro, 'f1_micro': f1_micro}

# ### binary
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
    
#     accuracy = accuracy_metric.compute(references=labels, predictions=predictions)['accuracy']
#     f1_true, f1_false = tuple(f1_metric.compute(references=labels, predictions=predictions, average=None, labels=[0,1])['f1'])
#     f1_macro = f1_metric.compute(references=labels, predictions=predictions, average='macro')['f1']
#     f1_micro = f1_metric.compute(references=labels, predictions=predictions, average='micro')['f1']
    
#     return {'accuracy': accuracy, 'f1_true': f1_true, 'f1_false': f1_false, 'f1_macro': f1_macro, 'f1_micro': f1_micro}

# Load Data

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["form"], examples["pair"], truncation=True)

In [13]:
train_dataset = pd.read_csv(TRAIN_DATA_PATH)
eval_dataset = pd.read_csv(EVAL_DATA_PATH)
train_dataset = datasets.Dataset.from_pandas(train_dataset).shuffle(seed=42)
eval_dataset = datasets.Dataset.from_pandas(eval_dataset).shuffle(seed=42)
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [14]:
len(train_dataset), len(eval_dataset)

(4391, 3002)

In [15]:
k = random.randrange(len(train_dataset))
print(tokenizer.decode(train_dataset['input_ids'][k]), train_dataset['labels'][k])
k = random.randrange(len(eval_dataset))
print(tokenizer.decode(eval_dataset['input_ids'][k]), eval_dataset['labels'][k])

[CLS] 역시 애정하는 아이소이 답다능👍 [SEP] 브랜드 # 일반 [SEP] 0
[CLS] 미세먼지까지 구석구석 잘 닦이고 # 자동물걸레청소기 라 힘들이지 않고도 쓱싹쓱싹 ~ 😆 [SEP] 본품 # 품질 [SEP] 0


# Load Trainer

In [16]:
args = TrainingArguments(
    output_dir=run_name,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,
    
    fp16=fp16,
)

In [17]:
# es = EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    # callbacks=[es],
)

# Run Trainer

In [19]:
trainer.train()
wandb.finish()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: form, pair, id. If form, pair, id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 4391
  Num Epochs = 12
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 2196
  Number of trainable parameters = 111093507
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Positive,F1 Negative,F1 Neutral,F1 Macro,F1 Micro
1,1.237200,0.106914,0.975017,0.987320,0.400000,0.000000,0.462440,0.975017
2,1.237200,0.101150,0.973018,0.988420,0.472222,0.066667,0.509103,0.973018
3,0.150700,0.111911,0.973018,0.987221,0.585366,0.255319,0.609302,0.973018
4,0.150700,0.121748,0.970686,0.986658,0.597015,0.219780,0.601151,0.970686
5,0.150700,0.143596,0.969021,0.985978,0.487805,0.278261,0.584015,0.969021
6,0.065800,0.134104,0.976016,0.988611,0.627451,0.171429,0.595830,0.976016
7,0.065800,0.196366,0.956362,0.978944,0.612245,0.248447,0.613212,0.956362
8,0.065800,0.164089,0.966023,0.984029,0.610169,0.278689,0.624296,0.966023
9,0.032900,0.148032,0.971352,0.986676,0.600000,0.260000,0.615559,0.971352
10,0.032900,0.149252,0.974017,0.988087,0.533333,0.216867,0.579429,0.974017


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: form, pair, id. If form, pair, id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 24
Saving model checkpoint to asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v23_run_1/checkpoint-183
Configuration saved in asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v23_run_1/checkpoint-183/config.json
Model weights saved in asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v23_run_1/checkpoint-183/pytorch_model.bin
tokenizer config file saved in asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v23_run_1/checkpoint-183/tokenizer_config.json
Special tokens file saved in asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v23_run_1/checkpoint-183/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argum

eval/accuracy,█▇▇▆▆█▁▄▆▇▇▇
eval/f1_macro,▁▃▇▇▆▇███▆▇▇
eval/f1_micro,█▇▇▆▆█▁▄▆▇▇▇
eval/f1_negative,▁▃▇▇▄██▇▇▅▇▇
eval/f1_neutral,▁▃▇▇█▅▇██▆▇▇
eval/f1_positive,▇█▇▇▆█▁▅▇█▇▇
eval/loss,▁▁▂▃▄▃█▆▄▅▅▅
eval/runtime,▁▇▁▁▇▁▁▁▁█▁▁
eval/samples_per_second,█▁██▂████▁██
eval/steps_per_second,█▁██▂████▁██
train/epoch,▁▂▂▃▃▃▄▄▄▅▆▆▆▇▇▇██


In [20]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(run_name)
for ckpt in ckpts:
    ckpt = os.path.join(run_name, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

!mv {run_name} {SAVE_PATH}/